In [2]:
import os

import matplotlib
matplotlib.use('AGG')

import matplotlib.pyplot as plt
import numpy as np

import keras

from keras.datasets import cifar10
from keras.layers import (Activation, Conv3D, Dense, Dropout, Flatten,
                          MaxPooling3D, MaxPooling2D)

from keras.layers import LeakyReLU
from keras.losses import categorical_crossentropy
from keras.models import Sequential
from keras.optimizers import Adam
#from keras.utils import np_utils
#from keras.utils.vis_utils import plot_model
from tensorflow.keras.utils import plot_model
from sklearn.model_selection import train_test_split

#import videoto3d
from tqdm import tqdm

from keras.callbacks import ModelCheckpoint
from keras.models import Model
from keras.layers import Input, Dense

##
import tensorflow as tf
from tensorflow.python.keras.backend import set_session
#from keras.backend.tensorflow_backend import set_session

In [3]:
import cv2


class Videoto3D:

    def __init__(self, width, height, depth):
        self.width = width
        self.height = height
        self.depth = depth

    def video3d(self, filename, color=False, skip=True):
        
        cap = cv2.VideoCapture(filename)
        
        nframe = cap.get(cv2.CAP_PROP_FRAME_COUNT)
        
        if skip:
            frames = [x * nframe / self.depth for x in range(self.depth)]
        else:
            frames = [x for x in range(self.depth)]
        
        framearray = []

        for i in range(self.depth):
            cap.set(cv2.CAP_PROP_POS_FRAMES, frames[i])
            ret, frame = cap.read()
        
            frame = cv2.resize(frame, (self.height, self.width))
        
            if color:
                framearray.append(frame)
            else:
                framearray.append(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY))

        cap.release()
        
        return np.array(framearray)

    def get_UCF_classname(self, filename):
        x =  filename[filename.find('_') + 1:filename.find('_', 2)]
        print(x)
        return x

In [4]:

def plot_history(history, result_dir):
    plt.plot(history.history['accuracy'], marker='.')
    plt.plot(history.history['val_accuracy'], marker='.')
    plt.title('model accuracy')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.grid()
    plt.legend(['accuracy', 'val_accuracy'], loc='lower right')
    plt.savefig(os.path.join(result_dir, 'model_accuracy.png'))
    plt.close()

    plt.plot(history.history['loss'], marker='.')
    plt.plot(history.history['val_loss'], marker='.')
    plt.title('model loss')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.grid()
    plt.legend(['loss', 'val_loss'], loc='upper right')
    plt.savefig(os.path.join(result_dir, 'model_loss.png'))
    plt.close()


def save_history(history, result_dir):
    loss = history.history['loss']
    acc = history.history['accuracy']
    val_loss = history.history['val_loss']
    val_acc = history.history['val_accuracy']
    nb_epoch = len(acc)

    with open(os.path.join(result_dir, 'result.txt'), 'w') as fp:
        fp.write('epoch\tloss\tacc\tval_loss\tval_acc\n')
        for i in range(nb_epoch):
            fp.write('{}\t{}\t{}\t{}\t{}\n'.format(
                i, loss[i], acc[i], val_loss[i], val_acc[i]))
    


def loaddata(video_dir, vid3d, nclass, result_dir, color=False, skip=True):
    
    files = os.listdir(video_dir)    
    X = []
    labels = []
    labellist = []

    pbar = tqdm(total=len(files))

    for filename in files:
    
        pbar.update(1)
        
        if filename == '.DS_Store':
            continue
        
        name = os.path.join(video_dir, filename)

        for v_files in os.listdir(name):

            v_file_path = os.path.join(name, v_files)

            label = vid3d.get_UCF_classname(filename)
            
            if label not in labellist:
                
                if len(labellist) >= nclass:
                    continue

                labellist.append(label)
            
            labels.append(label)
            
            X.append(vid3d.video3d(v_file_path, color=color, skip=skip))

    pbar.close()

    with open(os.path.join(result_dir, 'classes.txt'), 'w') as fp:
        for i in range(len(labellist)):
            fp.write('{}\n'.format(labellist[i]))

    for num, label in enumerate(labellist):
        for i in range(len(labels)):
            if label == labels[i]:
                labels[i] = num
                
    if color:
        return np.array(X).transpose((0, 2, 3, 4, 1)), labels
    else:
        return np.array(X).transpose((0, 2, 3, 1)), labels

In [5]:
def main():

    print('simple 3D convolution for action recognition')
    batch = 128
    # training epoch
    epoch = 100
    # directory where videos are stored
    videos= 'UCF50'
    # Number of classes
    nclass = 50
    output = 'results'
    color  = False
    skip   = True
    depth  = 10

    
    img_rows, img_cols, frames = 32, 32, depth
    channel = 3 if color else 1
    fname_npz = 'dataset_{}_{}_{}.npz'.format(nclass, depth, skip)

    vid3d = Videoto3D(img_rows, img_cols, frames)
    nb_classes = nclass
    if os.path.exists(fname_npz):
        loadeddata = np.load(fname_npz)
        X, Y = loadeddata["X"], loadeddata["Y"]
    else:
        x, y = loaddata(videos, vid3d, nclass,
                        output, color, skip)
        X = x.reshape((x.shape[0], img_rows, img_cols, frames, channel))
        Y = keras.utils.to_categorical(y, nb_classes)

        X = X.astype('float32')
        np.savez(fname_npz, X=X, Y=Y)
        print('Saved dataset to dataset.npz.')
    print('X_shape:{}\nY_shape:{}'.format(X.shape, Y.shape))

    # Define model
    # model = Sequential()


    # model.add(Conv3D(16, kernel_size=(3, 3, 3), input_shape=(
    #     X.shape[1:]), padding='same'))
    # model.add(LeakyReLU(alpha=.001)) 


    # model.add(Conv3D(32, kernel_size=(3, 3, 3), input_shape=(
    #     X.shape[1:]), padding='same'))
    # model.add(LeakyReLU(alpha=.001)) 

    ###########################

    input_x = Input(shape = (32, 32, depth, 1))

    initial_conv = Conv3D(16, kernel_size= (3, 3, 3), padding='same')(input_x)
    initial_conv = LeakyReLU(alpha=.001)(initial_conv)

    initial_conv = Conv3D(32, kernel_size= (3, 3, 3), padding='same')(initial_conv)
    initial_conv = LeakyReLU(alpha=.001)(initial_conv)


    ###########################
    # PARALLEL 1

    conv1 = Conv3D(16, kernel_size=(1, 1, 1),padding='same')(initial_conv)
    conv1 = LeakyReLU(alpha=.001)(conv1)
    conv1 = MaxPooling3D(pool_size=(2, 2, 2), padding='same')(conv1)


    # conv1 = Conv3D(16, kernel_size=(3, 3, 3),padding='same')(conv1)
    # conv1 = LeakyReLU(alpha=.001)(conv1)

    conv1 = Conv3D(16, kernel_size=(1, 1, 1),padding='same')(conv1)
    conv1 = LeakyReLU(alpha=.001)(conv1)

    
    conv1 = MaxPooling3D(pool_size=(2, 2, 2), padding='same')(conv1)

    # conv1 = Conv3D(8, kernel_size=(1, 1, 1),padding='same')(conv1)

    #check it    
    # conv1 = LeakyReLU(alpha=.001)(conv1)

    # conv1 = Conv3D(1, kernel_size=(1, 1, 1),padding='same')(conv1)

    ##############################

    ##############################

    #Parallel 2

    conv2 = Conv3D(8, kernel_size=(1, 1, 1),padding='same')(initial_conv)
    conv2 = LeakyReLU(alpha=.001)(conv2)
    conv2 = MaxPooling3D(pool_size=(2, 2, 2), padding='same')(conv2)


    # conv2 = Conv3D(8, kernel_size=(3, 3, 3),padding='same')(conv2)
    # conv2 = LeakyReLU(alpha=.001)(conv2)

    conv2 = Conv3D(16, kernel_size=(1, 1, 1),padding='same')(conv2)
    conv2 = LeakyReLU(alpha=.001)(conv2)
    

    conv2 = MaxPooling3D(pool_size=(2, 2, 2), padding='same')(conv2)

    # conv2 = Conv3D(4, kernel_size=(1, 1, 1),padding='same')(conv2)

    # #check it    
    # conv2 = LeakyReLU(alpha=.001)(conv2)

    # conv2 = Conv3D(1, kernel_size=(1, 1, 1),padding='same')(conv2)

    ###################################


    ##############################

    #Parallel 3

    conv3 = Conv3D(4, kernel_size=(1, 1, 1),padding='same')(initial_conv)
    conv3 = LeakyReLU(alpha=.001)(conv3)
    conv3 = MaxPooling3D(pool_size=(2, 2, 2), padding='same')(conv3)


    # conv3 = Conv3D(4, kernel_size=(3, 3, 3),padding='same')(conv3)
    # conv3 = LeakyReLU(alpha=.001)(conv3)

    conv3 = Conv3D(16, kernel_size=(1, 1, 1),padding='same')(conv3)
    conv3 = LeakyReLU(alpha=.001)(conv3)
    

    conv3 = MaxPooling3D(pool_size=(2, 2, 2), padding='same')(conv3)

    # conv3 = Conv3D(4, kernel_size=(1, 1, 1),padding='same')(conv3)

    # #check it    
    # conv3 = LeakyReLU(alpha=.001)(conv3)

    # conv3 = Conv3D(1, kernel_size=(1, 1, 1),padding='same')(conv3)

    ###################################

    added = keras.layers.Add()([conv1, conv2, conv3])

    added = MaxPooling3D(pool_size=(2, 2, 2), padding='same')(added)

    added = Flatten()(added)

    dense_1 = Dense(256)(added)

    dense_2 = Dense(nb_classes, activation = 'softmax')(dense_1)

    model = Model(input_x, dense_2)

    model.compile(loss=categorical_crossentropy,
                  optimizer=Adam(), metrics=['accuracy']) 

    model.summary()
    # plot_model(model, show_shapes=True,
    #            to_file=os.path.join(args.output, 'model.png'))

    X_train, X_test, Y_train, Y_test = train_test_split(
        X, Y, test_size=0.2, random_state=43)
    print(X_train.shape)

    ####################

    # 1
    filepath="d_3dcnnmodel-{epoch:02d}-{val_accuracy:.2f}.weights.hd5.keras"
    
    checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
    
    callbacks_list = [checkpoint]

    # 2 

    config = tf.compat.v1.ConfigProto()
    config.gpu_options.per_process_gpu_memory_fraction = 0.3
    set_session(tf.compat.v1.Session(config=config))

    ###############

    # Train model
    history = model.fit(X_train, Y_train, validation_data=(X_test, Y_test), batch_size=batch,
                        epochs=epoch, verbose=1, shuffle=True, callbacks=callbacks_list)
    
    model.evaluate(X_test, Y_test, verbose=0)
    
    model_json = model.to_json()
    
    if not os.path.isdir(output):
        os.makedirs(output)
    with open(os.path.join(output, 'ucf50_3dcnnmodel.json'), 'w') as json_file:
        json_file.write(model_json)
    
    model.save_weights(os.path.join(output, 'ucf50_3dcnnmodel-gpu.weights.h5'))

    loss, acc = model.evaluate(X_test, Y_test, verbose=0)
    
    print('Test loss:', loss)
    print('Test accuracy:', acc)

    plot_history(history, output)
    save_history(history, output)

In [6]:
os.chdir('D:\\Teach_n_Train\\Advanced_Lessons_CV\\LABS\\CV_3D_Data\\simple3D') 
if __name__ == '__main__':
    main()

simple 3D convolution for action recognition
X_shape:(6681, 32, 32, 10, 1)
Y_shape:(6681, 50)


C:\Users\WinIF Chung\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)      │ (None, 32, 32, 10, 1)     │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv3d (Conv3D)               │ (None, 32, 32, 10, 16)    │             448 │ input_layer[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ leaky_re_lu (LeakyReLU)       │ (None, 32, 32, 10, 16)    │               0 │ conv3d[0][0]               │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv3d_1 (Conv3D)             │ (None, 32, 32, 10, 32)    │          13,856 │ leaky_re_lu[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ leaky_re_lu_1 (LeakyReLU)     │ (None, 32, 32, 10, 32)    │               0 │ conv3d_1[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv3d_2 (Conv3D)             │ (None, 32, 32, 10, 16)    │             528 │ leaky_re_lu_1[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv3d_4 (Conv3D)             │ (None, 32, 32, 10, 8)     │             264 │ leaky_re_lu_1[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv3d_6 (Conv3D)             │ (None, 32, 32, 10, 4)     │             132 │ leaky_re_lu_1[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ leaky_re_lu_2 (LeakyReLU)     │ (None, 32, 32, 10, 16)    │               0 │ conv3d_2[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ leaky_re_lu_4 (LeakyReLU)     │ (None, 32, 32, 10, 8)     │               0 │ conv3d_4[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ leaky_re_lu_6 (LeakyReLU)     │ (None, 32, 32, 10, 4)     │               0 │ conv3d_6[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling3d (MaxPooling3D)  │ (None, 16, 16, 5, 16)     │               0 │ leaky_re_lu_2[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling3d_2               │ (None, 16, 16, 5, 8)      │               0 │ leaky_re_lu_4[0][0]        │
│ (MaxPooling3D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling3d_4               │ (None, 16, 16, 5, 4)      │               0 │ leaky_re_lu_6[0][0]        │
│ (MaxPooling3D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv3d_3 (Conv3D)             │ (None, 16, 16, 5, 16)     │             272 │ max_pooling3d[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv3d_5 (Conv3D)             │ (None, 16, 16, 5, 16)     │             144 │ max_pooling3d_2[0][0]      │
├───────────────────────────────┼───────────────────────────┼───────────────

 Total params: 159,902 (624.62 KB)

 Trainable params: 159,902 (624.62 KB)

 Non-trainable params: 0 (0.00 B)

(5344, 32, 32, 10, 1)

Epoch 1/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 628ms/step - accuracy: 0.0219 - loss: 28.6787
Epoch 1: val_accuracy improved from -inf to 0.03441, saving model to d_3dcnnmodel-01-0.03.weights.hd5.keras
42/42 ━━━━━━━━━━━━━━━━━━━━ 30s 670ms/step - accuracy: 0.0221 - loss: 28.3432 - val_accuracy: 0.0344 - val_loss: 4.3547
Epoch 2/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 626ms/step - accuracy: 0.0420 - loss: 4.0996
Epoch 2: val_accuracy improved from 0.03441 to 0.04562, saving model to d_3dcnnmodel-02-0.05.weights.hd5.keras
42/42 ━━━━━━━━━━━━━━━━━━━━ 28s 663ms/step - accuracy: 0.0422 - loss: 4.0964 - val_accuracy: 0.0456 - val_loss: 3.8504
Epoch 3/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 617ms/step - accuracy: 0.0767 - loss: 3.7450
Epoch 3: val_accuracy improved from 0.04562 to 0.07330, saving model to d_3dcnnmodel-03-0.07.weights.hd5.keras
42/42 ━━━━━━━━━━━━━━━━━━━━ 27s 652ms/step - accuracy: 0.0769 - loss: 3.7442 - val_accuracy: 0.0733 - val_loss: 3.7369
Epoch 4/100
42/42 ━━━━━━━━━━━━